In [46]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import json

In [47]:
results = pd.DataFrame({"Dataset": pd.Series(dtype='str'), "Budget": pd.Series(dtype='str'), "SM": pd.Series(dtype='float'), "ICP": pd.Series(dtype='float'), "IFN": pd.Series(dtype='float'), "NED": pd.Series(dtype='float')})
projects = ["JPetStore", "DayTrader", "AcmeAir", "Plants"]
for proj in projects:
	with open(f"results/Mo2oM/Mo2oM_{proj}.json") as f:
		data = json.load(f)
	df = pd.DataFrame(data).drop(columns=["microservices"])
	normalized_df = pd.DataFrame(MinMaxScaler().fit_transform(df.values), columns=df.columns)
	df['score'] = normalized_df["SM"] - normalized_df["ICP"] - normalized_df["IFN"] - normalized_df["NED"]
	final_df = pd.DataFrame({"Dataset": pd.Series(dtype='str'), "Budget": pd.Series(dtype='str'), "SM": pd.Series(dtype='float'), "ICP": pd.Series(dtype='float'), "IFN": pd.Series(dtype='float'), "NED": pd.Series(dtype='float')})
	s, m, l = np.array_split(df["n_clusters"].unique(), 3)
	budgets = {"Low": s, "Medium": m, "High": l}
	for budget, size in budgets.items():
		middle_df = pd.DataFrame(columns=["n_clusters", "SM", "ICP", "IFN", "NED"]).astype(dtype=df.drop(columns=["alpha", "threshold", "score"]).dtypes)
		for n in size:
			idx = df["score"][df["n_clusters"] == n].argmax()
			row = df[df["n_clusters"] == n].iloc[[idx]].drop(columns=["alpha", "threshold", "score"])
			middle_df = pd.concat([middle_df, row])
		row = middle_df.reset_index(drop=True).drop(columns=["n_clusters"]).mean().to_frame().T
		row["Budget"] = budget
		row["Dataset"] = proj
		final_df = pd.concat([final_df, row])
	results = pd.concat([results, final_df])
results

,Dataset,Budget,SM,ICP,IFN,NED
0,JPetStore,Low,0.327978,0.094626,3.174603,0.000000
0,JPetStore,Medium,0.294597,0.080190,1.185185,0.160657
0,JPetStore,High,0.237417,0.183848,0.986425,0.058519
0,DayTrader,Low,1.594964,0.055061,2.559252,0.109317
0,DayTrader,Medium,0.496922,0.071599,1.065012,0.160473
0,DayTrader,High,0.425931,0.148667,0.600938,0.211090
0,AcmeAir,Low,0.213381,0.090697,2.214286,0.008475
0,AcmeAir,Medium,0.329120,0.116488,0.888889,0.077799
0,AcmeAir,High,0.255410,0.132582,0.461538,0.107843
0,Plants,Low,0.811609,0.035696,3.375000,0.000000


In [49]:
index = pd.MultiIndex.from_frame(results[["Dataset", "Budget"]])
new_df = results.drop(columns=["Dataset", "Budget"]).set_index(index)
new_tex = new_df.to_latex(float_format="%.3f", escape=False, caption="Flexibility")
new_tex = new_tex.replace("[t]", "").replace("\\cline{1-6}", "\\midrule")
new_tex = new_tex.replace("SM", "\\textbf{SM $\\uparrow$}").replace("ICP", "\\textbf{ICP $\\downarrow$}").replace("IFN", "\\textbf{IFN $\\downarrow$}").replace("NED", "\\textbf{NED $\\downarrow$}")
new_tex = new_tex.replace("JPetStore", "\\textbf{JPetStore}").replace("DayTrader", "\\textbf{DayTrader}").replace("AcmeAir", "\\textbf{AcmeAir}").replace("Plants", "\\textbf{Plants}")
new_tex = new_tex.replace("Dataset", "\\textbf{Dataset}").replace("Budget", "\\textbf{Budget}")
print(new_tex)

\begin{table}
\caption{Flexibility}
\begin{tabular}{llrrrr}
\toprule
 &  & \textbf{SM $\uparrow$} & \textbf{ICP $\downarrow$} & \textbf{IFN $\downarrow$} & \textbf{NED $\downarrow$} \\
\textbf{Dataset} & \textbf{Budget} &  &  &  &  \\
\midrule
\multirow{3}{*}{\textbf{JPetStore}} & Low & 0.328 & 0.095 & 3.175 & 0.000 \\
 & Medium & 0.295 & 0.080 & 1.185 & 0.161 \\
 & High & 0.237 & 0.184 & 0.986 & 0.059 \\
\midrule
\multirow{3}{*}{\textbf{DayTrader}} & Low & 1.595 & 0.055 & 2.559 & 0.109 \\
 & Medium & 0.497 & 0.072 & 1.065 & 0.160 \\
 & High & 0.426 & 0.149 & 0.601 & 0.211 \\
\midrule
\multirow{3}{*}{\textbf{AcmeAir}} & Low & 0.213 & 0.091 & 2.214 & 0.008 \\
 & Medium & 0.329 & 0.116 & 0.889 & 0.078 \\
 & High & 0.255 & 0.133 & 0.462 & 0.108 \\
\midrule
\multirow{3}{*}{\textbf{Plants}} & Low & 0.812 & 0.036 & 3.375 & 0.000 \\
 & Medium & 0.513 & 0.080 & 0.938 & 0.000 \\
 & High & 0.563 & 0.120 & 0.267 & 0.042 \\
\midrule
\bottomrule
\end{tabular}
\end{table}

